In [70]:
# grader-required-cell

import os
import zipfile
import random
import shutil
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from shutil import copyfile
import matplotlib.pyplot as plt

In [71]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [72]:
root_dir = '/content/drive/Shareddrives/Capstone/MakeUpMate/DataGambar/labeled_image'
class_size = 10
start_from = 1

In [73]:
def create_train_val_dirs(root_path, class_size, start_from):

  os.makedirs(os.path.join(root_path, "Train"))
  os.makedirs(os.path.join(root_path, "Validation"))


  #Create train dirs
  for i in range(start_from, class_size+1):
    os.makedirs(os.path.join(root_path, "Train/" + f"{i}"))

  #Create validation dirs
  for i in range(start_from, class_size+1):
    os.makedirs(os.path.join(root_path, "Validation/" + f"{i}"))


In [74]:
try:
  create_train_val_dirs(root_path=root_dir, class_size=class_size, start_from=start_from)
except FileExistsError:
  print("You should not be seeing this since the upper directory is removed beforehand")

In [81]:
train_list_dir = os.listdir(os.path.join(root_dir, "Train"))
validation_list_dir = os.listdir(os.path.join(root_dir, "Validation"))
source_list_dir = os.listdir(os.path.join(root_dir))
source_list_dir.remove("Train")
source_list_dir.remove("Validation")

print("Train Directories:" + f"{train_list_dir}\n")
print("Validation Directories:" + f"{validation_list_dir}\n")
print("Source Directories:" + f"{source_list_dir}\n")

Train Directories:['1', '2', '3', '4', '5', '6', '7', '8', '9', '10']

Validation Directories:['1', '2', '3', '4', '5', '6', '7', '8', '9', '10']

Source Directories:['3', '1', '5', '7', '4', '6', '10', '2', '8', '9']



In [82]:
import math

def split_data(SOURCE_DIR, TRAINING_DIR, VALIDATION_DIR, SPLIT_SIZE):

  list_source = os.listdir(SOURCE_DIR)
  list_source_rand = random.sample(list_source, len(list_source))
  size_train = int(len(list_source)*SPLIT_SIZE)
  #split train
  for i in range(size_train):
    if os.path.getsize(os.path.join(SOURCE_DIR, list_source_rand[i])) == 0:
      print(f"{list_source_rand[i]} is zero length, so ignoring.")
      continue
    copyfile(os.path.join(SOURCE_DIR, list_source_rand[i]), os.path.join(TRAINING_DIR, list_source_rand[i]))

  for i in range(size_train, len(list_source)):
    if os.path.getsize(os.path.join(SOURCE_DIR, list_source_rand[i])) == 0:
      print(f"{list_source_rand[i]} is zero length, so ignoring.")
      continue
    copyfile(os.path.join(SOURCE_DIR, list_source_rand[i]), os.path.join(VALIDATION_DIR, list_source_rand[i]))


In [85]:
#Check if directory empty
for i in range(0, class_size):
  #Train
  if len(os.listdir(os.path.join(root_dir, "Train/"+f"{train_list_dir[i]}"))) > 0:
    for file in os.scandir(os.path.join(root_dir, "Train/"+f"{train_list_dir[i]}")):
      os.remove(file.path)
  #Validation
  if len(os.listdir(os.path.join(root_dir, "Validation/"+f"{validation_list_dir[i]}"))) > 0:
    for file in os.scandir(os.path.join(root_dir, "Validation/"+f"{validation_list_dir[i]}")):
      os.remove(file.path)

split_size = .8

# Run the function
for i in range(start_from, class_size+1):
  source_dir = os.path.join(root_dir, f"{i}")
  training_dir = os.path.join(root_dir, "Train/"+f"{i}")
  validation_dir = os.path.join(root_dir, "Validation/"+f"{i}")
  split_data(source_dir, training_dir, validation_dir, split_size)

# Check that the number of images matches the expected output

# Your function should perform copies rather than moving images so original directories should contain unchanged images
for i in range(start_from, class_size+1):
  source_dir = os.path.join(root_dir, f"{i}")
  print(f"\nOriginal {i} directory has {len(os.listdir(source_dir))} images")

# Training and validation splits
for i in range(start_from, class_size+1):
  training_dir = os.path.join(root_dir, "Train/"+f"{i}")
  validation_dir = os.path.join(root_dir, "Validation/"+f"{i}")
  print(f"\nTraining {i} directory has {len(os.listdir(training_dir))} images")
  print(f"Validation {i} directory has {len(os.listdir(validation_dir))} images")


Original 1 directory has 280 images

Original 2 directory has 164 images

Original 3 directory has 90 images

Original 4 directory has 186 images

Original 5 directory has 178 images

Original 6 directory has 165 images

Original 7 directory has 76 images

Original 8 directory has 143 images

Original 9 directory has 141 images

Original 10 directory has 111 images

Training 1 directory has 224 images
Validation 1 directory has 56 images

Training 2 directory has 131 images
Validation 2 directory has 33 images

Training 3 directory has 72 images
Validation 3 directory has 18 images

Training 4 directory has 148 images
Validation 4 directory has 38 images

Training 5 directory has 142 images
Validation 5 directory has 36 images

Training 6 directory has 132 images
Validation 6 directory has 33 images

Training 7 directory has 60 images
Validation 7 directory has 16 images

Training 8 directory has 114 images
Validation 8 directory has 29 images

Training 9 directory has 112 images
Vali

#Please Only Run From This Code Below For Machine Learning Purposes

In [86]:
training_dir = os.path.join(root_dir, "Train")
validation_dir = os.path.join(root_dir, "Validation")

In [87]:
def train_val_generators(TRAINING_DIR, VALIDATION_DIR):

  train_datagen = ImageDataGenerator(rescale=1./255.)

  train_generator = train_datagen.flow_from_directory(directory=TRAINING_DIR,
                                                      batch_size=20,
                                                      class_mode='binary',
                                                      target_size=(150, 150))

  validation_datagen = ImageDataGenerator(rescale=1./255.)
  # Pass in the appropriate arguments to the flow_from_directory method
  validation_generator = validation_datagen.flow_from_directory(directory=VALIDATION_DIR,
                                                                batch_size=20,
                                                                class_mode='binary',
                                                                target_size=(150, 150))
  ### END CODE HERE
  return train_generator, validation_generator

In [88]:
train_generator, validation_generator = train_val_generators(training_dir, validation_dir)

Found 1214 images belonging to 10 classes.
Found 311 images belonging to 10 classes.


In [89]:
def create_model():

  model = tf.keras.models.Sequential([ 
      tf.keras.layers.Conv2D(16, (3, 3), activation='relu', input_shape=(150, 150, 3)),
      tf.keras.layers.MaxPooling2D(2, 2),
      tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
      tf.keras.layers.MaxPooling2D(2, 2),
      tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
      tf.keras.layers.MaxPooling2D(2, 2),
      tf.keras.layers.Flatten(),
      tf.keras.layers.Dense(512, activation='relu'),
      tf.keras.layers.Dense(10, activation='softmax')
  ])

  
  model.compile(optimizer=tf.keras.optimizers.RMSprop(learning_rate=0.001),
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy']) 
    
  return model

In [90]:
# Get the untrained model
model = create_model()

# Train the model
# Note that this may take some time.
history = model.fit(train_generator,
                    epochs=10,
                    verbose=1,
                    validation_data=validation_generator)

Epoch 1/10
61/61 [==============================] - 365s 6s/step - loss: 1.6355 - accuracy: 0.4909 - val_loss: 0.5187 - val_accuracy: 0.8457
Epoch 2/10
61/61 [==============================] - 394s 6s/step - loss: 0.4975 - accuracy: 0.8328 - val_loss: 0.2945 - val_accuracy: 0.9003
Epoch 3/10
61/61 [==============================] - 366s 6s/step - loss: 0.2887 - accuracy: 0.9020 - val_loss: 0.1585 - val_accuracy: 0.9357
Epoch 4/10
61/61 [==============================] - 365s 6s/step - loss: 0.1795 - accuracy: 0.9415 - val_loss: 0.1814 - val_accuracy: 0.9325
Epoch 5/10
61/61 [==============================] - 361s 6s/step - loss: 0.1436 - accuracy: 0.9448 - val_loss: 0.2745 - val_accuracy: 0.9357
Epoch 6/10
61/61 [==============================] - 351s 6s/step - loss: 0.1384 - accuracy: 0.9465 - val_loss: 0.1782 - val_accuracy: 0.9421
Epoch 7/10
61/61 [==============================] - 356s 6s/step - loss: 0.0896 - accuracy: 0.9629 - val_loss: 0.1950 - val_accuracy: 0.9132
Epoch 8/10
61

In [98]:
import numpy as np

from google.colab import files
from tensorflow.keras.utils import load_img, img_to_array
import json
from json import JSONEncoder

import sys



uploaded=files.upload()

sample_arr = ""

for fn in uploaded.keys():
 
  path='/content/' + fn
  img=load_img(path, target_size=(150, 150))
  
  x=img_to_array(img)
  x /= 255
  x=np.expand_dims(x, axis=0)
  images = np.vstack([x])

  # np.set_printoptions(threshold=sys.maxsize)

  # arr_str = np.array_str(images)

  # # print(arr_str)


  # with open('readme.txt', 'w') as f:
  #     for line in images:
  #         f.write(str(line))
  #         f.write('\n')
  
  classes = model.predict(images, batch_size=10)
  
  print(classes[0])

class_result = 0
temp = 0
counter = 0

for i in classes[0]:
  counter+=1
  if i > temp:
    temp = i
    class_result = counter

print(class_result)
 

Saving LINE_ALBUM_Minggu 2 Januari_230125_11.jpg to LINE_ALBUM_Minggu 2 Januari_230125_11.jpg
1/1 [==============================] - 0s 33ms/step
[1.0544240e-02 7.1780100e-06 4.4280598e-01 1.9432115e-07 4.2723431e-03
 7.5122521e-06 5.4197377e-01 3.7371443e-04 1.5023150e-05 5.7337579e-09]
7


In [102]:
# Save the entire model as a SavedModel.
!mkdir -p saved_model
model.save('saved_model/my_model.h5')

In [103]:
!zip -r /content/saved_model.zip /content/saved_model
# change sample_data.zip to your desired download name Ex: nothing.zip
# change sample_data to your desired download folder name Ex: ner_data 

updating: content/saved_model/ (stored 0%)
updating: content/saved_model/my_model (deflated 8%)
  adding: content/saved_model/my_model.h5 (deflated 8%)


In [104]:
from google.colab import files
files.download("/content/saved_model.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>